In [ ]:
import itertools
import string
import random
from functools import reduce
import ipywidgets as widgets


random.seed(1287)
# global value indicating if the inference step required additional steps
inference_with_more_steps = False


def count_compatibility(item, pair_compatibility):
    """
    This function compute the compatibility of an item in the power set using a pair compatibility dictionary,
    which assign a compatibility value to each pair of items.

    Args:
        item: A specific item in the power set.
        pair_compatibility: A dictionary which assigns a compatibility value to each pair of elements in the power set.

    Returns:
        The sum of the compatibilities of each 2-combination of the elements in the item.
    """
    return sum(
        pair_compatibility[tuple(sorted(comb))]
        for comb in itertools.combinations(item, 2)
    )


def randomchain(p, q, n_atoms, cut=0, max_iter=500, mix_induction_abduction=True):
    """
    This function computes the random chain of inferences.

    Args:
        p: An iterable of items representing the Hypothesis.
        q: An iterable of items representing the Consequence.
        n_atoms: The number of atoms used to compute the power set.
        cut (int, optional):    A threshold used to reduce the size of the power set based on
                                the compatibility value of the elements in the power set. Defaults to 0.
        max_iter (int, optional):   Maximum number of iterations used to compute the inference chain.
                                    If the chain requires more than "max_iter" steps, then exit.
                                    Defaults to 500.
        mix_induction_abduction (bool, optional):   A boolean parameter used to compute a mixed chain of inductions and abductions.
                                                    If True, then the chain can be mixed. Otherwise, the chain is a sequence of
                                                    only inductions or only abductions. Defaults to True.

    Returns:
        _type_: _description_
    """
    global inference_with_more_steps

    # get the ASCII lowercase letters
    letters = list(string.ascii_lowercase)

    # get the atoms
    atoms = set(letters[:n_atoms])

    # compute a random pair compatibility dictionary
    # assign to every 2-combinations of the atoms a random compatibility value in [-1, 1]
    pair_compatibility = {
        tuple(sorted(comb)): (random.random() * 2 - 1)
        for comb in itertools.combinations(atoms, 2)
    }

    # compute the power set of the atoms
    thepowerset: list[list[str]] = reduce(
        lambda result, x: result + [subset + [x] for subset in result], atoms, [[]]
    )[1:]

    # reduce the power set by removing the items with compatibility value lower than "cut"
    thepowerset = [
        item
        for item in thepowerset
        if count_compatibility(item, pair_compatibility) >= cut
    ]

    hypothesis = set(p)
    consequence = set(q)
    iterations = 0
    inferences = []
    inference = None
    while not inference and iterations < max_iter:
        # compute the current inference (mixed or not) from the hypothesis to the consequence using the reduced power set
        inference = generate_inference(
            hypothesis,
            consequence,
            thepowerset,
            mix_induction_abduction=mix_induction_abduction,
        )
        if inference is not None:
            # no inference, then the chain reaches the final step
            iterations = 0
            inference = [sorted(element) for element in inference]
            inferences.extend(inference)
            inference = str(inference)
            print(pretty(inference))
        else:
            iterations += 1

    # if the inference chain required the maximum number of iterations,
    # then no inference was found from hypothesis to consequence
    if iterations == max_iter:
        print("Inference not found!")

    # boolean value indicating if the inference chain is only induction chain
    all_inducing = all(
        e[0] == "<" for e in inferences if len(set(e) & set([">", "<"])) == 1
    )
    # boolean value indicating if the inference chain is only abduction chain
    all_abducing = all(
        e[0] == ">" for e in inferences if len(set(e) & set([">", "<"])) == 1
    )
    return inferences, all_inducing, all_abducing, inference_with_more_steps


def generate_inference(
    hypothesis: set,
    consequence: set,
    thepowerset: list[list[str]],
    max_iter: int = 250,
    mix_induction_abduction: bool = True,
    max_attempts: int = 200,
):
    """
    This function compute the inference from the hypothesis to the consequence.

    Args:
        hypothesis (set): The set corresponding to the hypothesis.
        consequence (set): The set corresponding to the consequence.
        thepowerset (list[list[str]]): The power set of the atoms used to construct the inference chain.
        max_iter (int, optional): Maximum number of attempts used to compute the next step of the chain. Defaults to 250.
        mix_induction_abduction (bool, optional):   A boolean parameter used to compute a mixed chain of inductions and abductions.
                                                    If True, then the chain can be mixed. Otherwise, the chain is a sequence of
                                                    only inductions or only abductions. Defaults to True.
        max_attempts (int, optional): Maximum number of attemps used to compute the chain. Defaults to 200.

    Returns:
        _type_: _description_
    """

    global inference_with_more_steps

    attempts = 0
    inference_with_more_steps = False
    powerset: list[list[str]] = thepowerset[:]
    inference: list[set] = [hypothesis]

    # boolean value indicating if the next step will an induction step (True) or an abduction step (False)
    inducing = random.choice([False, True])

    while True:
        if attempts >= max_attempts:
            return None

        # boolean value indicating if the rule (induction or abduction) is found
        rule_matched = False

        last_element = set(inference[-1])
        element = set()
        listelement = list()

        iterations = 0
        while not rule_matched and iterations < max_iter:
            # trying to compute the next step of the inference chain in at most "max_iter" steps
            listelement = random.choice(powerset)
            element = set(listelement)
            if inducing:
                # for the induction, check if "last_element" is a proper subset of "element"
                rule_matched = last_element < element  # is proper subset
            else:
                # for the abduction, check if "last_element" is a proper superset of "element"
                rule_matched = last_element > element  # is proper superset
            iterations += 1

        if not rule_matched:
            # no rule found, then try if it is possible to find an inference step with the opposite rule (induction/abduction) of the current rule
            inducing = not inducing
            attempts += 1
            continue

        # use "<" for induction and ">" for abduction
        operator = "<" if inducing else ">"
        inference.extend([operator, element])
        # remove the element from the power set to avoid circular loops
        powerset.remove(listelement)

        # if the inference chain is a mixed chain of inductions and abductions, then change the rule
        if mix_induction_abduction:
            inducing = not inducing

        if element == consequence:
            # the element is the consequence, so exit
            return inference
        else:
            # try compute again the random chain
            can_go_on = False

            # check if it is possible to try to compute the inference chain with some more steps
            if inducing:
                # check if the last computed "element" is a proper subset of some element in the powerset
                can_go_on = any(element < set(myset) for myset in powerset)
            else:
                # check if the last computed "element" is a proper superset of some element in the powerset
                can_go_on = any(element > set(myset) for myset in powerset)

            inference_with_more_steps = can_go_on
            if not can_go_on:
                return None


def pretty(inference):
    """
    This function is used to compute a pretty textual version of an inference chain.
    """

    replacements = {
        "[[": "[",
        "]]": "]",
        "[": "{",
        "]": "}",
        ",": "+",
        "'": "",
        " ": "",
        "+{<}+": " < ",
        "+{>}+": " > ",
    }
    for key, value in replacements.items():
        inference = inference.replace(key, value)
    return inference


widgets.interact(
    randomchain,
    p="ad",
    q="acde",
    n_atoms=widgets.IntSlider(min=3, max=26, step=1, value=5),
    cut=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0),
    max_iter=widgets.IntSlider(min=10, max=1000, step=1, value=10),
    mix_induction_abduction=widgets.Checkbox(value=False),
)

interactive(children=(Text(value='ad', description='p'), Text(value='acde', description='q'), IntSlider(value=…

<function __main__.randomchain(p, q, n_atoms, cut=0, max_iter=500, mix_induction_abduction=True)>